In [1]:

from sklearn.datasets import fetch_openml
import numpy as np
from types import SimpleNamespace
from typing import Tuple, List
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
    
boston = fetch_openml(data_id = 531, data_home='./data_cache')

data = boston.data

label = pd.Series(boston.target)


category_cols = ["CHAS"]
continuous_cols = [x for x in data.columns if x not in category_cols]

le = LabelEncoder()
for col in category_cols:
    data[col] = le.fit_transform(data[col])


    
    
temp = None
for col in category_cols:
    oh_values = OneHotEncoder().fit_transform(data[col].values.reshape((-1, 1))).toarray()
    new_cols = [col + "-" + str(i) for i in range(len(data[col].unique()))]
    oh_values = pd.DataFrame(oh_values, columns = new_cols, dtype=np.int8, index=data.index)
    if temp is None:
        temp = oh_values
    else:
        temp = temp.merge(oh_values, left_index=True, right_index=True)

data = data.merge(temp, left_index=True, right_index=True)
data.drop(category_cols, inplace=True, axis=1)

category_cols = temp.columns

scaler = MinMaxScaler()
data[continuous_cols] = scaler.fit_transform(data[continuous_cols])

/opt/conda/envs/py38/lib/python3.8/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/tmp/ipykernel_395798/923413905.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])


In [2]:
import torch.nn as nn
model_hparams = {
    "encoder_dim" : data.shape[1],
    "predictor_hidden_dim" : 64,
    "predictor_output_dim" : 1,
    'alpha1' : 0.5,
    'alpha2' : 0.5,
    'beta' : 0.5,
    'K' : 10
}
data_hparams = {
    "K" : 10,
    "p_m" : 0.2
}
optim_hparams = {
    "lr" : 0.05
}
scheduler_hparams = {
    'gamma' : 0.3,
    'step_size' : 30
}
num_categoricals = len(continuous_cols)
num_continuous = len(continuous_cols)
consistency_loss = nn.MSELoss
loss_fn = nn.MSELoss
metric =  "mean_squared_error"
random_seed = 0

/opt/conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from misc.scorer import BaseScorer


class MSEScorer(BaseScorer):
    def __init__(self, metric: str) -> None:
        super().__init__(metric)
    
    def __call__(self, y, y_hat) -> float:
        return self.metric(y, y_hat)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(data, label, train_size = 0.7, random_state=random_seed)

X_train, X_unlabeled, y_train, _ = train_test_split(X_train, y_train, train_size = 0.1, random_state=random_seed)

In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from data_utils import *

gpus = [1]
n_jobs = 32
max_epochs = 30
batch_size = 64

pretraining_patience = 10
early_stopping_patience = 10

def fit_model(
            model,
            data_hparams
    ):
    
    train_ds = VIMESelfDataset(X_train.append(X_unlabeled), data_hparams, continuous_cols, category_cols)
    test_ds = VIMESelfDataset(X_valid, data_hparams, continuous_cols, category_cols)
    
    pl_datamodule = PLDataModule(train_ds, test_ds, batch_size=batch_size, is_regression=True)

    model.do_pretraining()

    callbacks = [
        EarlyStopping(
            monitor= 'val_loss', 
            mode = 'min',
            patience = pretraining_patience,
            verbose = False
        )
    ]
    pretraining_path = f'temporary_ckpt_data/pretraining'
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath=pretraining_path,
        filename='pretraining-{epoch:02d}-{val_f1:.4f}',
        save_top_k=1,
        mode = 'min'
    )

    callbacks.append(checkpoint_callback)

    trainer = Trainer(
                    devices = gpus,
                    accelerator="cuda" if len(gpus) >= 1 else 'cpu',
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = callbacks,
    )

    trainer.fit(model, pl_datamodule)
    
    pretraining_path = checkpoint_callback.best_model_path

    model = model.load_from_checkpoint(pretraining_path)

    model.do_finetunning()
    
        
    train_ds = VIMERegressionDataset(X_train, y_train.values, data_hparams, X_unlabeled, continuous_cols, category_cols)
    test_ds = VIMERegressionDataset(X_valid, y_valid.values, data_hparams, None, continuous_cols, category_cols)

    pl_datamodule = PLDataModule(train_ds, test_ds, batch_size = batch_size, is_regression=True, n_jobs=32)
        
    callbacks = [
        EarlyStopping(
            monitor= 'val_' + metric, 
            mode = 'min',
            patience = early_stopping_patience,
            verbose = False
        )
    ]

    checkpoint_path = None

    checkpoint_path = f'temporary_ckpt_data/'
    checkpoint_callback = ModelCheckpoint(
        monitor='val_' + metric,
        dirpath=checkpoint_path,
        filename='{epoch:02d}-{val_f1:.4f}',
        save_top_k=1,
        mode = 'min'
    )

    callbacks.append(checkpoint_callback)

    trainer = Trainer(
                    devices = gpus,
                    accelerator = "cuda" if len(gpus) >= 1 else 'cpu',
                    # replace_sampler_ddp=False,
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = callbacks,
    )

    trainer.fit(model, pl_datamodule)

    model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
    
    return model

In [6]:
hparams_range = {
    
    'predictor_hidden_dim' : ['suggest_int', ['predictor_hidden_dim', 16, 512]],
    # 'predictor_output_dim' : ['suggest_int', ['emb_dim', 16, 512]],
    
    'p_m' : ["suggest_float", ["p_m", 0.1, 0.9]],
    'alpha1' : ["suggest_float", ["alpha1", 0.1, 5]],
    'alpha2' : ["suggest_float", ["alpha2", 0.1, 5]],
    'beta' : ["suggest_float", ["beta", 0.0, 3]],
    'K' : ["suggest_int", ["K", 2, 20]],


    'lr' : ['suggest_float', ['lr', 0.0001, 0.05]],
    'gamma' : ['suggest_float', ['gamma', 0.1, 0.95]],
    'step_size' : ['suggest_int', ['step_size', 10, 100]],
}

In [7]:
import optuna
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

def objective(      trial: optuna.trial.Trial,
        ) -> float:
        """Objective function for optuna

        Args:
            trial: A object which returns hyperparameters of a model of hyperparameter search trial.
            train_idx: Indices of training data in self.data and self.label.
            test_idx: Indices of test data in self.data and self.label.
            fold_idx: A fold index that denotes which fold under the given k-fold cross validation.
        
        Returns:
            A score of given hyperparameters.
        """
        model_hparams = {
            "encoder_dim" : data.shape[1],
            "predictor_hidden_dim" : None,
            "predictor_output_dim" : 1,
            'alpha1' : None,
            'alpha2' : None,
            'beta' : None,
            'K' : None
        }
        
        data_hparams = {
            "K" : None,
            "p_m" : None
        }
        optim_hparams = {
            "lr" : None
        }
        scheduler_hparams = {
            'gamma' : None,
            'step_size' : None
        }

        for k, v in hparams_range.items():
            if k in model_hparams.keys():
                model_hparams[k] = getattr(trial, v[0])(*v[1])
            if k in data_hparams.keys():
                data_hparams[k] = getattr(trial, v[0])(*v[1])
            if k in optim_hparams.keys():
                optim_hparams[k] = getattr(trial, v[0])(*v[1])
            if k in scheduler_hparams.keys():
                scheduler_hparams[k] = getattr(trial, v[0])(*v[1])

        
        from pl_vime import PLVIME  
        pl_vime = PLVIME(model_hparams, "Adam", optim_hparams, "StepLR", scheduler_hparams, 
        num_categoricals, num_continuous, -1, loss_fn,
        MSEScorer("mean_squared_error"), random_seed)
        
        pl_vime = fit_model(pl_vime, data_hparams)
        

        trainer = Trainer(
                    devices = gpus,
                    accelerator = "cuda" if len(gpus)>= 1 else 'cpu',
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = None,
        )
        test_ds = VIMERegressionDataset(X_valid, y_valid.values, data_hparams, None, continuous_cols, category_cols)
        test_dl = DataLoader(test_ds, batch_size, shuffle=False, sampler = SequentialSampler(test_ds), num_workers=n_jobs)

        preds = trainer.predict(pl_vime, test_dl)

        preds = np.concatenate([out.cpu().numpy() for out in preds])

        from sklearn.metrics import mean_squared_error
        mse = mean_squared_error(y_valid, preds.squeeze())

        return mse

In [8]:
study = optuna.create_study(direction="minimize",sampler=optuna.samplers.TPESampler(seed=random_seed))
study.optimize(objective, n_trials=10, show_progress_bar=False)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")


trial = study.best_trial

print("  MSE: {}".format(trial.value))
print("  Best hyperparameters: ", trial)

[I 2023-12-19 15:30:24,562] A new study created in memory with name: no-name-723a4137-3f79-485b-ad74-402ba56ae847
Global seed set to 0
/tmp/ipykernel_395798/2137398929.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_ds = VIMESelfDataset(X_train.append(X_unlabeled), data_hparams, continuous_cols, category_cols)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/callb

/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 27: 100%|██████████| 6/6 [00:03<00:00,  1.80it/s, v_num=510, train_loss=34.10, val_loss=34.50]


Global seed set to 0
/workspace/vime/data_utils.py:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(unlabeled_data)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /workspace/vime/temporary_ckpt_data exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                      | Type             | Params
---------------------------------------------------------------
0 | model                     | VIME             | 88.5 K
1 | pretraining_mask_loss     | BCELoss          | 0     
2 | pretraining_feature_loss1 | CrossEntropyLoss | 0     
3 | pretraini

/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


...

Epoch 29: 100%|██████████| 6/6 [00:03<00:00,  1.73it/s, v_num=538, train_loss=23.20, train_mean_squared_error=16.10, val_mean_squared_error=33.90, val_loss=31.20]


Global seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 51.73it/s] 


[I 2023-12-19 16:03:38,983] Trial 9 finished with value: 33.86572824976366 and parameters: {'predictor_hidden_dim': 221, 'p_m': 0.1513179970790275, 'alpha1': 3.4931133849130975, 'alpha2': 2.8763471256122184, 'beta': 0.7961684728183362, 'K': 11, 'lr': 0.00478763148684624, 'gamma': 0.5895545212227524, 'step_size': 94}. Best is trial 5 with value: 32.349984983121324.


Number of finished trials:  10
Best trial:
  MSE: 32.349984983121324
  Best hyperparameters:  FrozenTrial(number=5, state=TrialState.COMPLETE, values=[32.349984983121324], datetime_start=datetime.datetime(2023, 12, 19, 15, 46, 49, 976058), datetime_complete=datetime.datetime(2023, 12, 19, 15, 50, 18, 474094), params={'predictor_hidden_dim': 349, 'p_m': 0.2683060488590727, 'alpha1': 0.7317388585087812, 'alpha2': 1.6455989195285012, 'beta': 1.0911323128278678, 'K': 12, 'lr': 0.021986215521769784, 'gamma': 0.9401177623503422, 'step_size': 19}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'predictor_hidden_dim': IntDistribution(high=512, log=False, low=16, step=1), 'p_m': FloatDistribution(high=0.9, log=False, low=0.1, step=None), 'alpha1': FloatDistribution(high=5.0, log=False, low=0.1, step=None), 'alpha2': FloatDistribution(high=5.0, log=False, low=0.1, step=None), 'beta': FloatDistribution(high=3.0, log=False, low=0.0, step=None), 'K': IntDistribution(high=20,

In [9]:
model_hparams = {
            "encoder_dim" : data.shape[1],
            "predictor_hidden_dim" : None,
            "predictor_output_dim" : 1,
            'alpha1' : None,
            'alpha2' : None,
            'beta' : None,
            'K' : None
        }
        
data_hparams = {
        "K" : None,
        "p_m" : None
}
optim_hparams = {
        "lr" : None
}
scheduler_hparams = {
        'gamma' : None,
        'step_size' : None
}

for k, v in study.best_trial.params.items():
        if k in model_hparams.keys():
                model_hparams[k] = study.best_trial.params[k]
        if k in data_hparams.keys():
                data_hparams[k] = study.best_trial.params[k]
        if k in optim_hparams.keys():
                optim_hparams[k] = study.best_trial.params[k]
        if k in scheduler_hparams.keys():
                scheduler_hparams[k] = study.best_trial.params[k]


from pl_vime import PLVIME
pl_vime = PLVIME(model_hparams, "Adam", optim_hparams, "StepLR", scheduler_hparams, 
       num_categoricals, num_continuous, -1, loss_fn,
       MSEScorer("mean_squared_error"), random_seed)
pl_vime = fit_model(pl_vime, data_hparams)

Global seed set to 0
/tmp/ipykernel_395798/2137398929.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_ds = VIMESelfDataset(X_train.append(X_unlabeled), data_hparams, continuous_cols, category_cols)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /workspace/vime/temporary_ckpt_data/pretraining exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                      | Type             | Params
---------------------------------------------------------------
0 | model                     | VIME             | 128 K 
1 | pretraining_mask_loss     | BCELoss

/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 29: 100%|██████████| 6/6 [00:03<00:00,  1.98it/s, v_num=540, train_loss=8.330, val_loss=8.520]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 6/6 [00:03<00:00,  1.98it/s, v_num=540, train_loss=8.330, val_loss=8.520]


...

/opt/conda/envs/py38/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 29: 100%|██████████| 6/6 [00:03<00:00,  1.73it/s, v_num=541, train_loss=17.30, train_mean_squared_error=8.180, val_mean_squared_error=38.50, val_loss=34.20]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 6/6 [00:03<00:00,  1.73it/s, v_num=541, train_loss=17.30, train_mean_squared_error=8.180, val_mean_squared_error=38.50, val_loss=34.20]


Global seed set to 0


In [10]:
trainer = Trainer(
                    devices = gpus,
                    accelerator = "cuda" if len(gpus) >= 1 else 'cpu',
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = None,
    )
test_ds = VIMERegressionDataset(X_valid, y_valid.values, data_hparams, None, continuous_cols, category_cols)
test_dl = DataLoader(test_ds, batch_size, shuffle=False, sampler = SequentialSampler(test_ds), num_workers=n_jobs)

preds = trainer.predict(pl_vime, test_dl)

preds = np.concatenate([out.cpu().numpy() for out in preds])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 76.57it/s] 


In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_valid, preds.squeeze())

32.349984983121324